In [ ]:
%%HTML <style>pre { white-space: pre !important; }</style>

In [ ]:
import org.apache.spark.sql.{SparkSession, DataFrame}

val spark = SparkSession.builder
        .appName("sesion_1")
        .master("local[*]")
        .getOrCreate()
val sc = spark.sparkContext

In [ ]:
sc.uiWebUrl

In [ ]:
val contractsDf = spark.read
        .option("header","true")
        .option("delimiter",",")
        .option("inferSchema","true")
        .csv("../../resources/data/csv/contracts.csv")

contractsDf.show(2)
contractsDf.printSchema()

#### Transformaciones

In [ ]:
// sort/orderBy

import org.apache.spark.sql.{functions => f}

contractsDf.orderBy(f.col("fec_alta").desc, f.col("cod_producto").asc).show(30)

In [ ]:
// dropDuplicates distinct

contractsDf.dropDuplicates(Seq("cod_producto", "activo")).show(2)

//contractsDf.distinct().show()

In [ ]:
// agg

contractsDf
    .agg(
        f.count("*").alias("count"),
        f.sum(f.col("cod_producto")).alias("sum_cod_producto"),
        f.max("cod_producto").alias("max_cod_producto"),
        f.min("cod_producto").alias("min_cod_producto"),
        f.mean("cod_producto").alias("mean_cod_producto"),
        f.avg("cod_producto").alias("avg_cod_producto"),
        f.approx_count_distinct("fec_alta", 0.05).alias("approx_fec_count"),
        f.count_distinct(f.col("fec_alta")).alias("distinct_fec_count"),
        f.max("fec_alta").alias("max_fec_alta")
).show()

In [ ]:
// groupBy -> count, max, min, avg, mean, sum

//contractsDf.groupBy("activo").count().show()
//contractsDf.groupBy(f.col("activo")).sum("cod_titular").show()
//contractsDf.groupBy(f.col("activo")).max("fec_alta").show()

contractsDf.groupBy(f.col("cod_iuc"), f.col("activo"))
    .agg(
        f.count("*").alias("count"),
        f.sum(f.col("cod_producto")).alias("sum_cod_producto"),
        f.max("cod_producto").alias("max_cod_producto"),
        f.min("cod_producto").alias("min_cod_producto"),
        f.mean("cod_producto").alias("mean_cod_producto"),
        f.avg("cod_producto").alias("avg_cod_producto"),
        f.max("fec_alta").alias("max_fec_alta")
    ).show()


In [ ]:
contractsDf
    .groupBy(f.col("cod_iuc"))
    .pivot("activo")
    .agg(
        f.count("*"),
        f.max("fec_alta")
    )
    .show()

In [ ]:
contractsDf
    .groupBy("cod_iuc")
    .pivot("activo")
    .agg(
        f.count("*"),
        f.max("fec_alta")
    )
    .unpivot(Array(f.col("cod_iuc")), Array(f.col("false_max(fec_alta)"), f.col("true_max(fec_alta)")), "metrica", "valor").show()

In [ ]:
// union/unionAll unionByName
val contracts_tmp_df = contractsDf.filter(f.col("activo") === "false")

contractsDf.show()

contractsDf.select("cod_titular","fec_alta","cod_producto")
    .unionAll(contractsDf.select("cod_producto","fec_alta","cod_titular")).show()


contractsDf.select("cod_titular","fec_alta","cod_producto", "activo")
    .unionByName(contractsDf.select("cod_producto","fec_alta","cod_titular"),  true).show()
